In [1]:
import os
os.chdir("D:/Flood_Project/Model_running/Nayeem")

In [2]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from scipy.spatial import cKDTree
import numpy as np

In [3]:
import time  # Import time library

# Start measuring time
start_time = time.time()

In [4]:
# Load shapefiles
polylines = gpd.read_file("path_travelled.shp")
points = gpd.read_file("CO_merged.shp")

In [5]:
# Ensure CRS is the same for both shapefiles
polylines = polylines.to_crs("EPSG:4326")
points = points.to_crs("EPSG:4326")


In [6]:
polylines.head(3)

,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,Lat_Cent,Long_Cent,geometry
0,4401341,5121,unclassified,Lyon Center Drive,None,F,24,0,F,F,42.5131,-83.6213,"LINESTRING (-83.62134 42.51291, -83.62132 42.5..."
1,4403731,5122,residential,Golden Valley Drive,None,B,0,0,F,F,42.5155,-83.6239,"LINESTRING (-83.62405 42.51520, -83.62387 42.5..."
2,4403735,5121,unclassified,New Hudson Drive,None,B,0,0,F,F,42.5111,-83.6215,"LINESTRING (-83.62176 42.51233, -83.62176 42.5..."


In [7]:
points.head(3)

,lat,lon,Row_Avg,geometry
0,42.025,-83.285,2118.789972,POINT (-83.28500 42.02500)
1,42.025,-83.283,2118.370450,POINT (-83.28300 42.02500)
2,42.025,-83.281,2117.938435,POINT (-83.28100 42.02500)


In [8]:
# Extract mid-point coordinates from polylines
if 'Lat_Cent' in polylines.columns and 'Long_Cent' in polylines.columns:
    poly_coords = np.array(polylines[['Lat_Cent', 'Long_Cent']])
else:
    # Calculate mid-point if not already provided
    polylines['mid_point'] = polylines.geometry.apply(lambda line: line.interpolate(0.5, normalized=True))
    polylines['Lat_Cent'] = polylines['mid_point'].y
    polylines['Long_Cent'] = polylines['mid_point'].x
    poly_coords = np.array(polylines[['Long_Cent', 'Lat_Cent']])

In [9]:
point_coords = np.array(points.geometry.apply(lambda geom: (geom.x, geom.y)).tolist())

In [10]:
# Build a KDTree for efficient nearest neighbor search
tree = cKDTree(point_coords)

In [11]:
# Query the nearest neighbor for each polyline mid-point
distances, indices = tree.query(poly_coords)

In [12]:
# Add the nearest point's attributes to the polylines
nearest_points = points.iloc[indices].reset_index(drop=True)
polylines = polylines.reset_index(drop=True)

In [13]:
# Copy attributes from nearest points
for col in points.columns:
    if col != 'geometry':  # Skip the geometry column
        polylines[f'nearest_{col}'] = nearest_points[col]

In [14]:
polylines['nearest_distance'] = distances

In [15]:
polylines

,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,Lat_Cent,Long_Cent,geometry,nearest_lat,nearest_lon,nearest_Row_Avg,nearest_distance
0,4401341,5121,unclassified,Lyon Center Drive,None,F,24,0,F,F,42.5131,-83.6213,"LINESTRING (-83.62134 42.51291, -83.62132 42.5...",42.025,-83.157,2202.494075,177.707531
1,4403731,5122,residential,Golden Valley Drive,None,B,0,0,F,F,42.5155,-83.6239,"LINESTRING (-83.62405 42.51520, -83.62387 42.5...",42.025,-83.157,2202.494075,177.711067
2,4403735,5121,unclassified,New Hudson Drive,None,B,0,0,F,F,42.5111,-83.6215,"LINESTRING (-83.62176 42.51233, -83.62176 42.5...",42.025,-83.157,2202.494075,177.706259
3,4413575,5114,secondary,West 8 Mile Road,MI 102,F,72,0,F,F,42.4419,-83.3220,"LINESTRING (-83.32216 42.44191, -83.32192 42.4...",42.025,-83.157,2202.494075,177.445637
4,4426485,5122,residential,Bradford Drive,None,F,0,0,F,F,42.5049,-83.6339,"LINESTRING (-83.63385 42.50516, -83.63387 42.5...",42.025,-83.157,2202.494075,177.710642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356916,1321444627,5141,service,None,None,B,0,0,F,F,42.5388,-83.1374,"LINESTRING (-83.13753 42.53877, -83.13751 42.5...",42.025,-83.157,2202.494075,177.383935
356917,1321444628,5141,service,None,None,B,0,0,F,F,42.5387,-83.1372,"LINESTRING (-83.13706 42.53873, -83.13728 42.5...",42.025,-83.157,2202.494075,177.383723
356918,1321444629,5141,service,None,None,B,0,0,F,F,42.5389,-83.1374,"LINESTRING (-83.13739 42.53890, -83.13739 42.5...",42.025,-83.157,2202.494075,177.384006
356919,1321444682,5141,service,None,None,F,0,0,F,F,42.5407,-83.1390,"LINESTRING (-83.13936 42.54064, -83.13868 42.5...",42.025,-83.157,2202.494075,177.386411


In [16]:
# Stop measuring time
end_time = time.time()

# Print runtime
print(f"Runtime: {end_time - start_time:.2f} seconds")

Runtime: 49.08 seconds
